## Hyperspectral Image Segmentation by Convolutional Neural Network 

### standardized spectra at full resolution with spatial dimension included

### training on north facing image and testing on two south facing

---

In [ ]:
# -- here are functions that generate a class that memory maps the raw data 
#    cube.  After executing this cell, the syntax is:
#    fname = "[path to data]/foo.raw"
#    cube = read_hyper(fname)

import os
import numpy as np

def read_header(hdrfile, verbose=True):
    """
    Read a Middleton header file.

    Parameters
    ----------
    hdrfile : str
        Name of header file.
    verbose : bool, optional
        If True, alert the user.

    Returns
    -------
    dict : dict
        A dictionary continaing the number of rows, columns, and wavelengths
        as well as an array of band centers.
    """

    # -- alert
    if verbose:
        print("reading and parsing {0}...".format(hdrfile))

    # -- open the file and read in the records
    recs = [rec for rec in open(hdrfile)]

    # -- parse for samples, lines, bands, and the start of the wavelengths
    for irec, rec in enumerate(recs):
        if 'samples' in rec:
            samples = int(rec.split("=")[1])
        elif 'lines' in rec:
            lines = int(rec.split("=")[1])
        elif 'bands' in rec:
            bands = int(rec.split("=")[1])
        elif "Wavelength" in rec:
            w0ind = irec+1

    # -- parse for the wavelengths
    waves = np.array([float(rec.split(",")[0]) for rec in 
                      recs[w0ind:w0ind+bands]])

    # -- return a dictionary
    return {"nrow":samples, "ncol":lines, "nwav":bands, "waves":waves}


def read_raw(rawfile, shape, hyper=False, verbose=True):
    """
    Read a Middleton raw file.

    Parameters
    ----------
    rawfile : str
        The name of the raw file.
    shape : tuple
        The output shape of the data cube (nwav, nrow, ncol).
    hyper : bool, optional
        Set this flag to read a hyperspectral image.
    verbose : bool, optional
        Alert the user.

    Returns
    -------
    memmap : memmap
        A numpy memmap of the datacube.
    """

    # -- alert
    if verbose:
        print("reading {0}...".format(rawfile))

    # -- read either broadband or hyperspectral image
    if hyper:
        return np.memmap(rawfile, np.uint16, mode="r") \
            .reshape(shape[2], shape[0], shape[1])[:, :, ::-1] \
            .transpose(1, 2, 0)
    else:
        return np.memmap(rawfile, np.uint8, mode="r") \
            .reshape(shape[1], shape[2], shape[0])[:, :, ::-1]


def read_hyper(fpath, fname=None, full=True):
    """
    Read a full hyperspectral scan (raw and header file).

    Parameters
    ----------
    fpath : str
        Either the full name+path of the raw file or the path of the raw file.
        If the latter, fname must be supplied.
    fname : str, optional
        The name of the raw file (required if fpath is set to a path).
    full : bool, optional
        If True, output a class containing data and supplementary information.
        If False, output only the data.

    Returns
    -------
    output or memmap : class or memmap
        If full is True, a class containing data plus supplementary 
        information.  If full is False, a memmap array of the data.
    """

    # -- set up the file names
    if fname is not None:
        fpath = os.path.join(fpath, fname)

    # -- read the header
    hdr = read_header(fpath.replace("raw", "hdr"))
    sh  = (hdr["nwav"], hdr["nrow"], hdr["ncol"])

    # -- if desired, only output data cube
    if not full:
        return read_raw(fpath, sh, hyper=True)

    # -- output full structure
    class output():
        def __init__(self, fpath):
            self.filename = fpath
            self.data     = read_raw(fpath, sh, hyper=True)
            self.waves    = hdr["waves"]
            self.nwav     = sh[0]
            self.nrow     = sh[1]
            self.ncol     = sh[2]

    return output(fpath)

In [ ]:
def kmeans_test_dictionary(labels, test, k):
    import collections, numpy
    
    unique_test, counts_test = numpy.unique(labels[test[:,0], test[:,1]], return_counts=True)
    counts_test_norm = (counts_test/test.shape[0])*100
    test_dict = dict(zip(unique_test, counts_test))
    test_dict_norm = dict(zip(unique_test, counts_test_norm))
    
    for i in range(0, k):
        if test_dict.get(i) is None: test_dict[i] = 0
        if test_dict_norm.get(i) is None: test_dict_norm[i] = 0
    
    return test_dict, test_dict_norm

In [ ]:
def kmeans_test_dataframe(sky_dict,
                          clouds_dict,
                          veg_dict,
                          wtr_dict,
                          blt_dict,
                          windows_dict,
                          rds_dict,
                          cars_dict,
                          mtl_dict
                         ):
    import pandas as pd
    
    pixel_names = ['sky', 'clouds', 'vegetation', 'water', 'built',
                  'windows', 'roads', 'cars', 'metal']
    df_test = pd.DataFrame([sky_dict,
                            clouds_dict,
                            veg_dict,
                            wtr_dict,
                            blt_dict,
                            windows_dict,
                            rds_dict,
                            cars_dict,
                            mtl_dict], index=pixel_names)
    df_test = df_test.transpose()
    
    return df_test

In [ ]:
def plot_confusion_matrix(df_test, norm=True):
    import numpy as np
    import matplotlib.pyplot as plt
    %matplotlib inline
    
    cm = np.array([df_test['sky'].values,
                   df_test['clouds'].values,
                   df_test['vegetation'].values,
                   df_test['water'].values,
                   df_test['built'].values,
                   df_test['windows'].values,
                   df_test['roads'].values,
                   df_test['cars'].values,
                   df_test['metal'].values])
    classes = ['sky', 'clouds', 'vegetation', 'water', 'built', 'windows', 'roads', 
              'cars', 'metal']
    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    if norm:
        title='Normalized Confusion Matrix'
        fmt='.2f'
    else:
        title='Confusion Matrix'
        fmt='d'
    ax.set(xticks=np.arange(cm.shape[1]),
          yticks=np.arange(cm.shape[0]),
          xticklabels=np.arange(0,cm.shape[1]).astype(str), 
          yticklabels=classes,
          title=title,
          ylabel='True Label',
          xlabel='Predicted Label')
    #plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
    #        rotation_mode="anchor")
    thresh = cm.max()/2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i,j], fmt),
                   ha="center", va="center",
                   color="white" if cm[i,j] > thresh else "black")
    fig.tight_layout()
    plt.show()

In [ ]:
def plot_test_result(df_Test):
    import matplotlib.pyplot as plt
    %matplotlib inline
    
    df_test = df_Test.transpose()
    ax = df_test.plot.bar(rot=0, stacked=True, colormap='tab20b')
                          #color=['tab:blue', 'tab:green', 'tab:gray'])
    plt.xlabel('Actual Class')
    plt.ylabel('%of Test Pixels')
    plt.title('Error in Kmeans Prediction')
    plt.legend(bbox_to_anchor=(1,1), loc=2, borderaxespad=1.0, prop={'size':11})
    plt.show()

In [ ]:
def point_from_string(text):
    
    items = text.strip("\n").split(" ")
    rind = int(items[0])
    cind = int(items[1])
    
    return rind, cind

In [ ]:
def coords(row, col):
    return np.array(list(np.ndindex((row, col)))).reshape(row, col, 2)

---
### Classes:
1. Sky
2. Clouds
3. Water
4. Vegetation
5. Buildings (concrete structures)
6. Windows
7. Roads
8. Cars
9. Metal Structures
---


## CNN Trained on North Facing Image

In [ ]:
fname_north = "../../image_files/scan1_slow_roof_VNIR.raw"
cube_north = read_hyper(fname_north)

In [ ]:
cube_sub_north = cube_north.data[:, :, :].astype(float)
print(cube_sub_north.shape)

In [ ]:
cube_reshaped_north = cube_sub_north.transpose(1, 2, 0).reshape((cube_sub_north.shape[1] * cube_sub_north.shape[2]), cube_sub_north.shape[0])
print(cube_reshaped_north.shape)

In [ ]:
cube_norm_north = (cube_reshaped_north - cube_reshaped_north.min()) / (cube_reshaped_north.max() - cube_reshaped_north.min())

import matplotlib.pyplot as plt
%matplotlib inline

red_ind_n = (np.abs(cube_north.waves - 650.0)).argmin()
green_ind_n = (np.abs(cube_north.waves - 550.0)).argmin()
blue_ind_n = (np.abs(cube_north.waves - 450.0)).argmin()

cube_reshaped_north2 = cube_norm_north.reshape(cube_sub_north.shape[1], cube_sub_north.shape[2], cube_sub_north.shape[0])
cube_scene_north = cube_reshaped_north2[:, :, [red_ind_n, green_ind_n, blue_ind_n]]
fig, ax = plt.subplots(figsize=(10,10))
plt.title('North Facing RGB Image')
ax.imshow(cube_scene_north, aspect=0.4)
plt.show()

In [ ]:
rgbn = cube_reshaped_north2[:, :, [red_ind_n, green_ind_n, blue_ind_n]].copy()
rgbn /= rgbn.mean((0, 1), keepdims=True)

fig, ax = plt.subplots(figsize=(10,10))
plt.title('North Facing corrected RGB Image')
ax.imshow(rgbn.clip(0, 1)**0.5, aspect=0.4)
plt.show()

In [ ]:
cube_standard_north = (cube_reshaped_north - cube_reshaped_north.mean(1, keepdims=True)) / cube_reshaped_north.std(1, keepdims=True)
cube_reshaped_north = cube_standard_north

In [ ]:
print(len(cube_north.waves))
print(min(cube_north.waves), max(cube_north.waves))
print(cube_reshaped_north.shape)

In [ ]:
# create position array and normalize

xycoordsn = coords(cube_sub_north.shape[1], cube_sub_north.shape[2])
xycoordsn = xycoordsn/xycoordsn.max()

In [ ]:
# append row, col position matrix to spectral data

cube_tempn = cube_reshaped_north.reshape(cube_sub_north.shape[1], cube_sub_north.shape[2], cube_reshaped_north.shape[1])
cube_specxyn = np.append(cube_tempn, xycoordsn, axis=2)

In [ ]:
# reshape spectral and spatial cube for clustering
cube_specxyn_2d = cube_specxyn.reshape((cube_specxyn.shape[0] * cube_specxyn.shape[1]), cube_specxyn.shape[2])
print(cube_specxyn_2d.shape)

#### Reading manually classified set for training and testing

In [ ]:
# read manually selected coordinates files

#sky coordinates
sky_filen = open("../manual_classified_pixels/1_sky_coordinates_north.txt", "r")
sky_coordsn = sky_filen.readlines()
sky_filen.close()
sky_coordsn = np.array([point_from_string(line) for line in sky_coordsn])
print("sky:        ", sky_coordsn.shape)

#clouds coordinates
clouds_filen = open("../manual_classified_pixels/2_clouds_coordinates_north.txt", "r")
clouds_coordsn = clouds_filen.readlines()
clouds_filen.close()
clouds_coordsn = np.array([point_from_string(line) for line in clouds_coordsn])
print("clouds:     ", clouds_coordsn.shape)

#vegetation coordinates
veg_filen = open("../manual_classified_pixels/3_vegetation_coordinates_north.txt", "r")
veg_coordsn = veg_filen.readlines()
veg_filen.close()
veg_coordsn = np.array([point_from_string(line) for line in veg_coordsn])
print("vegetation: ", veg_coordsn.shape)

#water coordinates
wtr_filen = open("../manual_classified_pixels/4_water_coordinates_north.txt", "r")
wtr_coordsn = wtr_filen.readlines()
wtr_filen.close()
wtr_coordsn = np.array([point_from_string(line) for line in wtr_coordsn])
print("water:      ", wtr_coordsn.shape)

#buildings coordinates
blt_filen = open("../manual_classified_pixels/5_buildings_coordinates_north.txt", "r")
blt_coordsn = blt_filen.readlines()
blt_filen.close()
blt_coordsn = np.array([point_from_string(line) for line in blt_coordsn])
print("buildings:  ", blt_coordsn.shape)

#windows coordinates
windows_filen = open("../manual_classified_pixels/6_windows_coordinates_north.txt", "r")
windows_coordsn = windows_filen.readlines()
windows_filen.close()
windows_coordsn = np.array([point_from_string(line) for line in windows_coordsn])
print("windows:    ", windows_coordsn.shape)

#roads coordinates
rds_filen = open("../manual_classified_pixels/7_roads_coordinates_north.txt", "r")
rds_coordsn = rds_filen.readlines()
rds_filen.close()
rds_coordsn = np.array([point_from_string(line) for line in rds_coordsn])
print("road:       ", rds_coordsn.shape)

#cars coordinates
cars_filen = open("../manual_classified_pixels/8_cars_coordinates_north.txt", "r")
cars_coordsn = cars_filen.readlines()
cars_filen.close()
cars_coordsn = np.array([point_from_string(line) for line in cars_coordsn])
print("cars:       ", cars_coordsn.shape)

#metal coordinates
mtl_filen = open("../manual_classified_pixels/9_metal_coordinates_north.txt", "r")
mtl_coordsn = mtl_filen.readlines()
mtl_filen.close()
mtl_coordsn = np.array([point_from_string(line) for line in mtl_coordsn])
print("metal:      ", mtl_coordsn.shape)

#### split classified pixels into 80% training and 20% testing sets

In [ ]:
import random

# sky coordinates
sky_indn = np.arange(sky_coordsn.shape[0])
random.Random(3).shuffle(sky_indn)
lim_ind = int(len(sky_indn)*0.8)
sky_train_indn = sky_indn[:lim_ind]
sky_test_indn = sky_indn[lim_ind:]
print("sky %d %d" % (len(sky_train_indn), len(sky_test_indn)))

# clouds coordinates
clouds_indn = np.arange(clouds_coordsn.shape[0])
random.Random(3).shuffle(clouds_indn)
lim_ind = int(len(clouds_indn)*0.8)
clouds_train_indn = clouds_indn[:lim_ind]
clouds_test_indn = clouds_indn[lim_ind:]
print("clouds %d %d" % (len(clouds_train_indn), len(clouds_test_indn)))

# vegetation coordinates
veg_indn = np.arange(veg_coordsn.shape[0])
random.Random(3).shuffle(veg_indn)
lim_ind = int(len(veg_indn)*0.8)
veg_train_indn = veg_indn[:lim_ind]
veg_test_indn = veg_indn[lim_ind:]
print("vegetation %d %d" % (len(veg_train_indn), len(veg_test_indn)))

# water coordinates
wtr_indn = np.arange(wtr_coordsn.shape[0])
random.Random(3).shuffle(wtr_indn)
lim_ind = int(len(wtr_indn)*0.8)
wtr_train_indn = wtr_ind[:lim_ind]
wtr_test_indn = wtr_ind[lim_ind:]
print("water %d %d" % (len(wtr_train_indn), len(wtr_test_indn)))

# built coordinates
blt_indn = np.arange(blt_coordsn.shape[0])
random.Random(3).shuffle(blt_indn)
lim_ind = int(len(blt_indn)*0.8)
blt_train_indn = blt_indn[:lim_ind]
blt_test_indn = blt_indn[lim_ind:]
print("built %d %d" % (len(blt_train_indn), len(blt_test_indn)))

# windows coordinates
windows_indn = np.arange(windows_coordsn.shape[0])
random.Random(3).shuffle(windows_indn)
lim_ind = int(len(windows_indn)*0.8)
windows_train_indn = windows_indn[:lim_ind]
windows_test_indn = windows_indn[lim_ind:]
print("windows %d %d" % (len(windows_train_indn), len(windows_test_indn)))

# roads coordinates
rds_indn = np.arange(rds_coordsn.shape[0])
random.Random(3).shuffle(rds_indn)
lim_ind = int(len(rds_indn)*0.8)
rds_train_indn = rds_indn[:lim_ind]
rds_test_indn = rds_indn[lim_ind:]
print("roads %d %d" % (len(rds_train_indn), len(rds_test_indn)))

# cars coordinates
cars_indn = np.arange(cars_coordsn.shape[0])
random.Random(3).shuffle(cars_indn)
lim_ind = int(len(cars_indn)*0.8)
cars_train_indn = cars_indn[:lim_ind]
cars_test_indn = cars_indn[lim_ind:]
print("cars %d %d" % (len(cars_train_indn), len(cars_test_indn)))

# metal coordinates
mtl_indn = np.arange(mtl_coordsn.shape[0])
random.Random(3).shuffle(mtl_indn)
lim_ind = int(len(mtl_indn)*0.8)
mtl_train_indn = mtl_indn[:lim_ind]
mtl_test_indn = mtl_indn[lim_ind:]
print("metal %d %d" % (len(mtl_train_indn), len(mtl_test_indn)))

In [ ]:
cube_sky_trainn = cube_specxyn[sky_coordsn[sky_train_indn[:], 0], sky_coordsn[sky_train_indn[:], 1], :]
cube_sky_testn = cube_specxyn[sky_coordsn[sky_test_indn[:], 0], sky_coordsn[sky_test_indn[:], 1], :]
print("sky ", cube_sky_trainn.shape, cube_sky_testn.shape)

cube_clouds_trainn = cube_specxyn[clouds_coordsn[clouds_train_indn[:], 0], clouds_coordsn[clouds_train_indn[:], 1], :]
cube_clouds_testn = cube_specxyn[clouds_coordsn[clouds_test_indn[:], 0], clouds_coordsn[clouds_test_indn[:], 1], :]
print("clouds ", cube_clouds_trainn.shape, cube_clouds_testn.shape)

cube_veg_trainn = cube_specxyn[veg_coordsn[veg_train_indn[:], 0], veg_coordsn[veg_train_indn[:], 1], :]
cube_veg_testn = cube_specxyn[veg_coordsn[veg_test_indn[:], 0], veg_coordsn[veg_test_indn[:], 1], :]
print("vegetation ", cube_veg_trainn.shape, cube_veg_testn.shape)

cube_wtr_trainn = cube_specxyn[wtr_coordsn[wtr_train_indn[:], 0], wtr_coordsn[wtr_train_indn[:], 1], :]
cube_wtr_testn = cube_specxyn[wtr_coordsn[wtr_test_indn[:], 0], wtr_coordsn[wtr_test_indn[:], 1], :]
print("water ", cube_wtr_trainn.shape, cube_wtr_testn.shape)

cube_blt_trainn = cube_specxyn[blt_coordsn[blt_train_indn[:], 0], blt_coordsn[blt_train_indn[:], 1], :]
cube_blt_testn = cube_specxyn[blt_coordsn[blt_test_indn[:], 0], blt_coordsn[blt_test_indn[:], 1], :]
print("built ", cube_blt_trainn.shape, cube_blt_testn.shape)

cube_windows_trainn = cube_specxyn[windows_coordsn[windows_train_indn[:], 0], windows_coordsn[windows_train_indn[:], 1], :]
cube_windows_testn = cube_specxyn[windows_coordsn[windows_test_indn[:], 0], windows_coordsn[windows_test_indn[:], 1], :]
print("windows ", cube_windows_trainn.shape, cube_windows_testn.shape)

cube_rds_trainn = cube_specxyn[rds_coordsn[rds_train_indn[:], 0], rds_coordsn[rds_train_indn[:], 1], :]
cube_rds_testn = cube_specxyn[rds_coordsn[rds_test_indn[:], 0], rds_coordsn[rds_test_indn[:], 1], :]
print("roads ", cube_rds_trainn.shape, cube_rds_testn.shape)

cube_cars_trainn = cube_specxyn[cars_coordsn[cars_train_indn[:], 0], cars_coordsn[cars_train_indn[:], 1], :]
cube_cars_testn = cube_specxyn[cars_coordsn[cars_test_indn[:], 0], cars_coordsn[cars_test_indn[:], 1], :]
print("cars ", cube_cars_trainn.shape, cube_cars_testn.shape)

cube_mtl_trainn = cube_specxyn[mtl_coordsn[mtl_train_indn[:], 0], mtl_coordsn[mtl_train_indn[:], 1], :]
cube_mtl_testn = cube_specxyn[mtl_coordsn[mtl_test_indn[:], 0], mtl_coordsn[mtl_test_indn[:], 1], :]
print("metal ", cube_mtl_trainn.shape, cube_mtl_testn.shape)

In [ ]:
# concatenate training and testing sets and create index arrays

cube_trainn = np.concatenate((cube_sky_trainn, cube_clouds_trainn, cube_veg_trainn, cube_blt_trainn, cube_wtr_trainn,
                            cube_windows_trainn, cube_rds_trainn, cube_cars_trainn, cube_mtl_trainn), axis=0)
cube_train_labelsn = [0]*cube_sky_trainn.shape[0] + [1]*cube_clouds_trainn.shape[0] \
                    + [2]*cube_veg_trainn.shape[0] + [3]*cube_wtr_trainn.shape[0] + [4]*cube_blt_trainn.shape[0] \
                    + [5]*cube_windows_trainn.shape[0] + [6]*cube_rds_trainn.shape[0] \
                    + [7]*cube_cars_trainn.shape[0] + [8]*cube_mtl_trainn.shape[0]

print(cube_trainn.shape)

cube_testn = np.concatenate((cube_sky_testn, cube_clouds_testn, cube_veg_testn, cube_blt_testn, cube_wtr_testn,
                            cube_windows_testn, cube_rds_testn, cube_cars_testn, cube_mtl_testn), axis=0)
cube_test_labelsn = [0]*cube_sky_testn.shape[0] + [1]*cube_clouds_testn.shape[0] \
                    + [2]*cube_veg_testn.shape[0] + [3]*cube_wtr_testn.shape[0] + [4]*cube_blt_testn.shape[0] \
                    + [5]*cube_windows_testn.shape[0] + [6]*cube_rds_testn.shape[0] \
                    + [7]*cube_cars_testn.shape[0] + [8]*cube_mtl_testn.shape[0]

print(cube_testn.shape)

#### Train CNN

In [ ]:
import tensorflow as tf
from tensorflow import keras

cnnn = keras.Sequential([keras.layers.Conv1D(32, kernel_size=(5), padding="same", 
                                            activation="relu", input_shape=(cube_specxyn.shape[2], 1)),
                       keras.layers.MaxPooling1D((2), strides=2),
                        keras.layers.Conv1D(64, kernel_size=(5), padding="same", activation="relu"),
                        keras.layers.MaxPooling1D((2), strides=2),
                        keras.layers.Flatten(),
                        keras.layers.Dense(1024, activation="relu"),
                        keras.layers.Dense(10, activation="softmax")])

cnnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy",
           metrics=["accuracy"])

cube_trainn2 = cube_trainn.reshape(cube_trainn.shape[0], cube_trainn.shape[1], 1)
cube_testn2 = cube_testn.reshape(cube_testn.shape[0], cube_testn.shape[1], 1)

CNNmodeln = cnnn.fit(cube_trainn2, cube_train_labelsn, epochs=50, batch_size=512)

In [ ]:
# summarize history of loss

f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(12, 10))
ax1.plot(CNNmodeln.history['loss'])
ax1.plot(CNNmodeln.history['val_loss'])
ax1.set_title('CNN Model Loss')
ax1.set_ylabel('loss')
ax1.set_xlabel('epoch')
ax1.locator_params(nbins=13, axis='x')
ax1.legend(['train', 'test'], loc='center right')
ax2.plot(CNNmodeln.history['accuracy'])
ax2.plot(CNNmodeln.history['val_accuracy'])
ax2.set_title('CNN Model Accuracy')
ax2.set_ylabel('accuracy')
ax2.set_xlabel('epoch')
ax2.locator_params(nbins=12, axis='x')
ax2.legend(['train', 'test'], loc='center right')
ax3.plot(CNNmodeln.history['loss'])
ax3.plot(CNNmodeln.history['val_loss'])
ax3.set_ylabel('log(loss)')
ax3.set_xlabel('epoch')
ax3.locator_params(nbins=13, axis='x')
ax3.legend(['train', 'test'], loc='center right')
ax3.set_yscale('log')
ax4.plot(CNNmodeln.history['accuracy'])
ax4.plot(CNNmodeln.history['val_accuracy'])
ax4.set_ylabel('log(accuracy)')
ax4.set_xlabel('epoch')
ax4.locator_params(nbins=12, axis='x')
ax4.legend(['train', 'test'], loc='center right')
ax4.set_yscale('log')
plt.show
f.savefig("./plots/CNN_spatial_train_north_1_north_CNN_loss_accuracy_vs_epoch.png")

In [ ]:
import tensorflow as tf
from tensorflow import keras

cnnn = keras.Sequential([keras.layers.Conv1D(32, kernel_size=(5), padding="same", 
                                            activation="relu", input_shape=(cube_specxyn.shape[2], 1)),
                       keras.layers.MaxPooling1D((2), strides=2),
                        keras.layers.Conv1D(64, kernel_size=(5), padding="same", activation="relu"),
                        keras.layers.MaxPooling1D((2), strides=2),
                        keras.layers.Flatten(),
                        keras.layers.Dense(1024, activation="relu"),
                        keras.layers.Dense(10, activation="softmax")])

cnnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy",
           metrics=["accuracy"])

cube_trainn2 = cube_trainn.reshape(cube_trainn.shape[0], cube_trainn.shape[1], 1)
cube_testn2 = cube_testn.reshape(cube_testn.shape[0], cube_testn.shape[1], 1)

CNNmodeln = cnnn.fit(cube_trainn2, cube_train_labelsn, epochs=20, batch_size=512)

In [ ]:
# model accuracy on training and testing sets

train_loss, train_acc = cnnn.evaluate(cube_trainn2, cube_train_labelsn)
test_loss, test_acc = cnnn.evaluate(cube_testn2, cube_test_labelsn)

print(train_acc, test_acc)

### Predict pixel classification on north facing image

In [ ]:
import time
start_time = time.time()

cube_specxyn_2d_1 = cube_specxyn_2d.reshape(cube_specxyn_2d.shape[0], cube_specxyn_2d.shape[1], 1)

predictCuben = cnnn.predict_classes(cube_specxyn_2d_1)

elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [ ]:
predictCube_reshapen = predictCuben.reshape(cube_sub_north.shape[1], cube_sub_north.shape[2])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as ListedColorMap
import matplotlib.patches as mpatches
from matplotlib.ticker import NullFormatter
%matplotlib inline

cmap = {0:[0,0.32549,0.62353,1], 1:[0.93333,0.9098,0.77255,1], 2:[0,0.61961,0.45098,1],  3:[0.33725,0.70588,0.91373,1],
        4:[0,0,0,1], 5:[1,0.82353,0,1], 6:[0.90196,0.62353,0,1], 7:[0.83529,0.36863,0,1],
        8:[0.8,0.47451,0.65490,1]}
labels = {0:'sky', 1:'clouds', 2:'vegetation', 3:'water', 4:'built',
          5:'windows', 6:'roads', 7:'cars', 8:'metal'}
arrayShow = np.array([[cmap[i] for i in j] for j in predictCube_reshapen])
patches = [mpatches.Patch(color=cmap[i], label=labels[i]) for i in cmap]

fig = plt.figure(figsize=(15,15))
ax = fig.add_axes([0.1,0.1,0.9,0.9])
ax.tick_params(labelsize=10)
ax.imshow(arrayShow, aspect=0.4)
lgd = ax.legend(handles=patches, bbox_to_anchor=(1,0.75), loc='upper left', borderaxespad=1.0, prop={'size':10}, ncol=1)
plt.show()
fig.savefig("./plots/CNN_spatial_train_north_2_north_predict_map.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
veg_by_row_north = np.zeros(cube_sub_north.shape[1])
for row in range(0, cube_sub_north.shape[1]):
    veg_by_row_north[row] = np.count_nonzero(predictCube_reshapen[row,:] == 2)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as ListedColorMap
import matplotlib.patches as mpatches
from matplotlib.ticker import NullFormatter
%matplotlib inline

cmap = {0:[0,0.32549,0.62353,1], 1:[0.93333,0.9098,0.77255,1], 2:[0,0.61961,0.45098,1],  3:[0.33725,0.70588,0.91373,1],
        4:[0,0,0,1], 5:[1,0.82353,0,1], 6:[0.90196,0.62353,0,1], 7:[0.83529,0.36863,0,1],
        8:[0.8,0.47451,0.65490,1]}
labels = {0:'sky', 1:'clouds', 2:'vegetation', 3:'water', 4:'built',
          5:'windows', 6:'roads', 7:'cars', 8:'metal'}
arrayShow = np.array([[cmap[i] for i in j] for j in predictCube_reshapen])
patches = [mpatches.Patch(color=cmap[i], label=labels[i]) for i in cmap]
#fig, ax = plt.subplots(figsize = (20,10))

fig = plt.figure(1, figsize=(30,10))
axImage = plt.axes([0.1,0.1,0.85,0.95])
axHist = plt.axes([0.85,0.1,0.2,0.95])
axHist.yaxis.set_major_formatter(NullFormatter())
axImage.tick_params(labelsize=20)
axHist.tick_params(labelsize=20)
axImage.imshow(arrayShow, aspect=0.5)
axHist.plot(veg_by_row_north, np.arange(0,cube_sub.shape[1]), color=[0.0,0.33,0.62])
axHist.fill_between(veg_by_row_north, np.arange(0,cube_sub_north.shape[1]), cube_sub_north.shape[1], facecolor=[0.0,0.33,0.62])
axHist.set_ylim(cube_sub_north.shape[1], 0)
axHist.set(title='Vegetation Pixels by Row')
axHist.title.set_fontsize(25)
lgd = axImage.legend(handles=patches, bbox_to_anchor=(0,0.5), loc=2, borderaxespad=-13.0, prop={'size':20}, ncol=1)
plt.show()
#fig.savefig("./output/plots/19_kmeans_clustering_of_veg_00108.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# plot mean spectra for testing set separately

import matplotlib.pyplot as plt
%matplotlib inline

f, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9)) = plt.subplots(3, 3, figsize=(12, 10))
plt.suptitle('Mean Standardized Spectra of Classified Pixels (north) vs Testing Pixels (north) vs Training Pixels (north)')
ax1.plot(cube_north.waves, cube_sky_trainn[:,:-2].mean(0), color=[0.170,0,0,0.75], lw=1, ls="dashed")
ax1.plot(cube_north.waves, cube_sky_testn[:,:-2].mean(0), color=[0.170,0,0,0.5], lw=1, ls="dotted")
ax1.plot(cube_north.waves, cube_reshaped_north[np.where(predictCuben == 0)[0]].mean(0), color=[0,0.32549,0.62353])
ax1.set_title("Sky")
ax1.legend(['train', 'test', 'predict'], loc='upper right')
ax2.plot(cube_north.waves, cube_clouds_trainn[:,:-2].mean(0), color=[0.170,0,0,0.75], lw=1, ls="dashed")
ax2.plot(cube_north.waves, cube_clouds_testn[:,:-2].mean(0), color=[0.170,0,0,0.5], lw=1, ls="dotted")
ax2.plot(cube_north.waves, cube_reshaped_north[np.where(predictCuben == 1)[0]].mean(0), color=[0.93333,0.9098,0.77255])
ax2.set_title("Clouds")
ax2.legend(['train', 'test', 'predict'], loc='upper right')
ax3.plot(cube_north.waves, cube_veg_trainn[:,:-2].mean(0), color=[0.170,0,0,0.75], lw=1, ls="dashed")
ax3.plot(cube_north.waves, cube_veg_testn[:,:-2].mean(0), color=[0.170,0,0,0.5], lw=1, ls="dotted")
ax3.plot(cube_north.waves, cube_reshaped_north[np.where(predictCuben == 2)[0]].mean(0), color=[0,0.61961,0.45098])
ax3.set_title("Vegetation")
ax3.legend(['train', 'test', 'predict'], loc='upper right')
ax4.plot(cube_north.waves, cube_wtr_trainn[:,:-2].mean(0), color=[0.170,0,0,0.75], lw=1, ls="dashed")
ax4.plot(cube_north.waves, cube_wtr_testn[:,:-2].mean(0), color=[0.170,0,0,0.5], lw=1, ls="dotted")
ax4.plot(cube_north.waves, cube_reshaped_north[np.where(predictCuben == 3)[0]].mean(0), color=[0.33725,0.70588,0.91373])
ax4.set_title("Water")
ax4.legend(['train', 'test', 'predict'], loc='upper right')
ax5.plot(cube_north.waves, cube_blt_trainn[:,:-2].mean(0), color=[0.170,0,0,0.75], lw=1, ls="dashed")
ax5.plot(cube_north.waves, cube_blt_testn[:,:-2].mean(0), color=[0.170,0,0,0.5], lw=1, ls="dotted")
ax5.plot(cube_north.waves, cube_reshaped_north[np.where(predictCuben == 4)[0]].mean(0), color=[0,0,0])
ax5.set_title("Built")
ax5.legend(['train', 'test', 'predict'], loc='upper right')
ax6.plot(cube_north.waves, cube_windows_trainn[:,:-2].mean(0), color=[0.170,0,0,0.75], lw=1, ls="dashed")
ax6.plot(cube_north.waves, cube_windows_testn[:,:-2].mean(0), color=[0.170,0,0,0.5], lw=1, ls="dotted")
ax6.plot(cube_north.waves, cube_reshaped_north[np.where(predictCuben == 5)[0]].mean(0), color=[1,0.82353,0])
ax6.set_title("Windows")
ax6.legend(['train', 'test', 'predict'], loc='upper right')
ax7.plot(cube_north.waves, cube_rds_trainn[:,:-2].mean(0), color=[0.170,0,0,0.75], lw=1, ls="dashed")
ax7.plot(cube_north.waves, cube_rds_testn[:,:-2].mean(0), color=[0.170,0,0,0.5], lw=1, ls="dotted")
ax7.plot(cube_north.waves, cube_reshaped_north[np.where(predictCuben == 6)[0]].mean(0), color=[0.90196,0.62353,0])
ax7.set_title("Roads")
ax7.legend(['train', 'test', 'predict'], loc='upper right')
ax8.plot(cube_north.waves, cube_cars_trainn[:,:-2].mean(0), color=[0.170,0,0,0.75], lw=1, ls="dashed")
ax8.plot(cube_north.waves, cube_cars_testn[:,:-2].mean(0), color=[0.170,0,0,0.5], lw=1, ls="dotted")
ax8.plot(cube_north.waves, cube_reshaped_north[np.where(predictCuben == 7)[0]].mean(0), color=[0.83529,0.36863,0])
ax8.set_title("Cars")
ax8.legend(['train', 'test', 'predict'], loc='upper right')
ax9.plot(cube_north.waves, cube_mtl_trainn[:,:-2].mean(0), color=[0.170,0,0,0.75], lw=1, ls="dashed")
ax9.plot(cube_north.waves, cube_mtl_testn[:,:-2].mean(0), color=[0.170,0,0,0.5], lw=1, ls="dotted")
ax9.plot(cube_north.waves, cube_reshaped_north[np.where(predictCuben == 8)[0]].mean(0), color=[0.8,0.47451,0.65490])
ax9.set_title("Metal")
ax9.legend(['train', 'test', 'predict'], loc='upper right')
plt.show()
f.savefig("./plots/CNN_spatial_train_north_3_north_mean_spectra_train_test_predict_3x3.png")

### Evaluation Metrics (north facing)

In [ ]:
labelsn_pred_sky = predictCube_reshapen[sky_coordsn[:,0], sky_coordsn[:,1]]
labelsn_pred_clouds = predictCube_reshapen[clouds_coordsn[:,0], clouds_coordsn[:,1]]
labelsn_pred_veg = predictCube_reshapen[veg_coordsn[:,0], veg_coordsn[:,1]]
labelsn_pred_wtr = predictCube_reshapen[wtr_coordsn[:,0], wtr_coordsn[:,1]]
labelsn_pred_blt = predictCube_reshapen[blt_coordsn[:,0], blt_coordsn[:,1]]
labelsn_pred_windows = predictCube_reshapen[windows_coordsn[:,0], windows_coordsn[:,1]]
labelsn_pred_rds = predictCube_reshapen[rds_coordsn[:,0], rds_coordsn[:,1]]
labelsn_pred_cars = predictCube_reshapen[cars_coordsn[:,0], cars_coordsn[:,1]]
labelsn_pred_mtl = predictCube_reshapen[mtl_coordsn[:,0], mtl_coordsn[:,1]]

labelsn_pred = np.concatenate((labelsn_pred_sky, labelsn_pred_clouds, labelsn_pred_veg, labelsn_pred_wtr, 
                             labelsn_pred_blt, labelsn_pred_windows, labelsn_pred_rds, labelsn_pred_cars, labelsn_pred_mtl))

In [ ]:
labelsn_true_sky = np.full((sky_coordsn.shape[0]), 0)
labelsn_true_clouds = np.full((clouds_coordsn.shape[0]), 1)
labelsn_true_veg = np.full((veg_coordsn.shape[0]), 2)
labelsn_true_wtr = np.full((wtr_coordsn.shape[0]), 3)
labelsn_true_blt = np.full((blt_coordsn.shape[0]), 4)
labelsn_true_windows = np.full((windows_coordsn.shape[0]), 5)
labelsn_true_rds = np.full((rds_coordsn.shape[0]), 6)
labelsn_true_cars = np.full((cars_coordsn.shape[0]), 7)
labelsn_true_mtl = np.full((mtl_coordsn.shape[0]), 8)

labelsn_true = np.concatenate((labelsn_true_sky, labelsn_true_clouds, labelsn_true_veg, labelsn_true_wtr, 
                             labelsn_true_blt, labelsn_true_windows, labelsn_true_rds, labelsn_true_cars, labelsn_true_mtl))

In [ ]:
print(labelsn_pred.shape)
print(labelsn_true.shape)

In [ ]:
from sklearn import metrics

print("Confusion Matrix")
print(metrics.confusion_matrix(labelsn_true, labelsn_pred))

print("")
print("Accuracy Score")
print(metrics.accuracy_score(labelsn_true, labelsn_pred))

print("")
print("Classification Report")
print(metrics.classification_report(labelsn_true, labelsn_pred))

In [ ]:
f = open("./metrics/CNN_spatial_train_north_test_north_metrics.txt", "w")
f.write("Confusion Matrix")
f.write('\n')
f.write(str(metrics.confusion_matrix(labelsn_true, labelsn_pred)))
f.write('\n')
f.write('\n')
f.write("Accuracy Score")
f.write('\n')
f.write(str(metrics.accuracy_score(labelsn_true, labelsn_pred)))
target_names = ['sky', 'clouds', 'vegetation', 'water', 'built', 'windows', 'roads', 'cars', 'metal']
f.write('\n')
f.write('\n')
f.write("Classification Report")
f.write('\n')
f.write(metrics.classification_report(labelsn_true, labelsn_pred, target_names=target_names))
f.close()

In [ ]:
sky_dictn, sky_dict_normn = kmeans_test_dictionary(predictCube_reshapen, sky_coordsn, 9)
clouds_dictn, cloud_dict_normn = kmeans_test_dictionary(predictCube_reshapen, clouds_coordsn, 9)
veg_dictn, veg_dict_normn = kmeans_test_dictionary(predictCube_reshapen, veg_coordsn, 9)
wtr_dictn, wtr_dict_normn = kmeans_test_dictionary(predictCube_reshapen, wtr_coordsn, 9)
blt_dictn, blt_dict_normn = kmeans_test_dictionary(predictCube_reshapen, blt_coordsn, 9)
windows_dictn, windows_dict_normn = kmeans_test_dictionary(predictCube_reshapen, windows_coordsn, 9)
rds_dictn, rds_dict_normn = kmeans_test_dictionary(predictCube_reshapen, rds_coordsn, 9)
cars_dictn, cars_dict_normn = kmeans_test_dictionary(predictCube_reshapen, cars_coordsn, 9)
mtl_dictn, mtl_dict_normn = kmeans_test_dictionary(predictCube_reshapen, mtl_coordsn, 9)

df_testn = kmeans_test_dataframe(sky_dictn, clouds_dictn, veg_dictn, wtr_dictn,
                                blt_dictn, windows_dictn, rds_dictn, cars_dictn, mtl_dictn)
print(df_testn.transpose())
df_test_normn = kmeans_test_dataframe(sky_dictn, clouds_dictn, veg_dictn, wtr_dictn,
                                     blt_dictn, windows_dictn, rds_dictn, cars_dictn, mtl_dictn)
#print("")
#print(df_test_normn.transpose())

In [ ]:
plot_confusion_matrix(df_testn, norm=False)
#plot_confusion_matrix(df_test_normn, norm=True)
plot_test_result(df_test_normn)

#### CNN Features